# Analysis of FFS Sampling



In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import MDAnalysis as mda
import os
import glob
import re
from matplotlib.ticker import ScalarFormatter

### OP Calculator

In [ ]:
def op(gro,xtc,offset=0,cutoff=-1,skip=1):
    system = mda.Universe(gro,xtc)
    F = system.select_atoms("resname F")
    Li = system.select_atoms("resname Li")
    dis = []
    time = []
    for ts in system.trajectory[offset:cutoff:skip]:
        time.append(system.trajectory.time)
        F_pos = F.positions[0]
        Li_pos = Li.positions[0]
        box = system.dimensions[0:3]
        x = abs(Li_pos[0]-F_pos[0])
        y = abs(Li_pos[1]-F_pos[1])
        z = abs(Li_pos[2]-F_pos[2])
        dx = min(x,abs(box[0]-x))
        dy = min(y,abs(box[1]-y))
        dz = min(z,abs(box[2]-z))
        dis.append((dx**2+dy**2+dz**2)**0.5)
    return dis

In [ ]:
MPATH = "/ocean/projects/see220002p/$(whoami)/LiF/"

### Plot the OP of Trajectories from Interfaces along with the OP from Previous Trajectories

In [ ]:
interface = 0
val = 10
string = "/sim_"

for j in np.linspace(0,999,val,dtype=int):
    lmda = []
    out = [[],[]]
    for i in np.linspace(interface,0,interface+1,dtype=int):
        if i == interface:
            for f in os.listdir(MPATH+"simulations/int_"+str(i)+string+str(j)):
                name, ext = os.path.splitext(f)
                if ext == '.gro':
                    config = f
                elif ext == '.xtc':
                    traj = f
            new = op(MPATH+"simulations/int_"+str(i)+string+str(j)+"/"+config,MPATH+"simulations/int_"+str(i)+string+str(j)+"/"+traj)
            lmda = new + lmda
            out[0] += [new[-1]]
            out[1] += [(len(new)-1)*0.0005]
        else:
            back = [int(float(thing)) for thing in re.findall(r'-?\d+\.?\d*', f)][2]
            for f in os.listdir(MPATH+"simulations/int_"+str(i)+"/sim_"+str(back)):
                name, ext = os.path.splitext(f)
                if ext == '.gro':
                    config = f
                elif ext == '.xtc':
                    traj = f
            new = op(MPATH+"simulations/int_"+str(i)+"/sim_"+str(back)+"/"+config,MPATH+"simulations/int_"+str(i)+"/sim_"+str(back)+"/"+traj)
            lmda = new + lmda
            out[0] += [new[-1]]
            for n in range(len(out[1])):
                out[1][n] += len(new)*0.0005
            out[1] += [(len(new)-1)*0.0005]
                
    time = [0.0005*t for t in range(len(lmda))]
    plt.plot(time,lmda,color='b')
    #plt.axhline(y=1.923518185874568,color='r')
    #plt.axhline(y=2.0982887958982905,color='k')
    #plt.axhline(y=2.1341968367524933,color='k')
    #plt.axhline(y=2.174052009069744,color='k')
    # plt.axhline(y=2.2889527234967133,color='k')
    # plt.axhline(y=2.3132094710880198,color='k')
    # plt.axhline(y=2.33320947108802,color='k')
    # plt.axhline(y=2.35320947108802,color='k')
    # plt.axhline(y=2.3220327558485665,color='k')
    # plt.axhline(y=2.3771194310662995,color='k')
    # plt.axhline(y=2.418057220368871,color='k')
    # plt.axhline(y=2.4629120938914326,color='k')
    # plt.axhline(y=2.5038129408292504,color='k')
    # plt.axhline(y=2.549677833958526,color='k')
    # plt.axhline(y=2.5913156167769045,color='k')
    # plt.axhline(y=2.659983674813393,color='k')
    # plt.axhline(y=2.7191041106917613,color='k')
    # plt.axhline(y=2.791750709402397,color='k')
    # plt.axhline(y=2.884419756628912,color='k')
    # plt.axhline(y=2.9628847099896096,color='k')
    # plt.axhline(y=3.0890101424840974,color='k')
    # plt.axhline(y=3.2248217807396307,color='k')
    # plt.axhline(y=3.4179820828165206,color='k')
    #plt.axhline(y=4.0,color='k')
    #plt.axhline(y=4.0,color='b')
    plt.plot(out[1],out[0],'ro')
    plt.show()
    
    

### Plot Groupsize Diagrams for Each Interface

In [ ]:
num = interface+1
for i in np.linspace(1,num,num,dtype=int):#range(1,4):
    configs = []
    for f in os.listdir(MPATH+"simulations/int_"+str(i)):
        name, ext = os.path.splitext(f)
        if ext == '.gro':
            configs += [f]
    length = len(configs)
    s = [None] * len(configs)
    for j in range(len(configs)):
        s[j] = [int(float(n)) for n in re.findall(r'-?\d+\.?\d*', configs[j])]
    end = i
    groups = [[] for m in range(end)]
    groupnum = [[] for m in range(end)] 
    for n in range(1,end+1):
        for j in range(length):
            for step in range(n):
                if step == 0:
                    for f in os.listdir(MPATH+"simulations/int_"+str(i-step-1)+"/sim_"+str(s[j][4])):
                        name, ext = os.path.splitext(f)
                        if ext == '.gro':
                            nums = [int(float(thing)) for thing in re.findall(r'-?\d+\.?\d*', f)]
                else:
                    for f in os.listdir(MPATH+"simulations/int_"+str(i-step-1)+"/sim_"+str(nums[2])):
                        name, ext = os.path.splitext(f)
                        if ext == '.gro':
                            nums = [int(float(thing)) for thing in re.findall(r'-?\d+\.?\d*', f)]
            inGroups = False
            for k in range(len(groups[n-1])):
                if groups[n-1][k] == nums[3]:
                    inGroups = True
                    groupnum[n-1][k] += 1/length
            if inGroups == False:
                groups[n-1] += [nums[3]]
                groupnum[n-1] += [1/length]
    fig, ax = plt.subplots()
    plt.yscale("log")
    plt.ylim(0.75/length,1)
    ax.yaxis.set_major_formatter(ScalarFormatter())
    bplotn = plt.boxplot(groupnum)
    plt.show()